In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

# commento Gianluca

In [ ]:
import numpy as np # linear algebra
#import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# import os
import matplotlib.pyplot as plt
import seaborn as sns
import math
import copy

#we need to use the follwing nflrush
# from kaggle.competitions import nflrush


from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

%matplotlib inline

## import keras
## import tensorflow as tf

In [ ]:
train = pd.read_csv('../input/nfl-big-data-bowl-2020/train.csv', dtype={'WindSpeed': 'object'})
train.shape #check numb or rows and cols

In [ ]:
# check variables with their types (int, object, etc)
train.info()

### Variables meaning:

- GameId - a unique game identifier
- PlayId - a unique play identifier
- Team - home or away
- X - player position along the long axis of the field. See figure below.
- Y - player position along the short axis of the field. See figure below.
- S - speed in yards/second
- A - acceleration in yards/second^2
- Dis - distance traveled from prior time point, in yards
- Orientation - orientation of player (deg)
- Dir - angle of player motion (deg)
- NflId - a unique identifier of the player
- Season - year of the season
- YardLine - the yard line of the line of scrimmage
- Quarter - game quarter (1-5, 5 == overtime)
- GameClock - time on the game clock
- PossessionTeam - team with possession
- Down - the down (1-2-3-4)
- Distance - yards needed for a first down
- FieldPosition - which side of the field the play is happening on
- HomeScoreBeforePlay - home team score before play started
- VisitorScoreBeforePlay - visitor team score before play started
- NflIdRusher - the NflId of the rushing player (importante!)
- OffenseFormation - offense formation
- OffensePersonnel - offensive team positional grouping
- DefendersInTheBox - number of defenders lined up near the line of scrimmage, spanning the width of the offensive line
- DefensePersonnel - defensive team positional grouping
- PlayDirection - direction the play is headed (left/right)
- TimeHandoff - UTC time of the handoff
- TimeSnap - UTC time of the snap
- Yards - the yardage gained on the play (you are predicting this/TARGET VAR)
- Week - week into the season
- StadiumType - description of the stadium environment
- Turf - description of the field surface
- GameWeather - description of the game weather
- Temperature - temperature (deg F)
- Humidity - humidity
- WindSpeed - wind speed in miles/hour
- WindDirection - wind direction

In [ ]:
pd.set_option('display.max_columns', None) #in order to display all the var/columns
train.head(10) #display first 10 rows/units

In [ ]:
# vedere quante diverse tipologia di forazioni off si posson avere (sicuramente possiam fare encoding dopo)
train.OffenseFormation.value_counts(ascending=True)

In [ ]:
# 'Yards' variable
train.Yards.describe()

In [ ]:
# Distribution plot (per Yards)
ax = sns.distplot(train['Yards'])
plt.vlines(train['Yards'].mean(), plt.ylim()[0], plt.ylim()[1], color='r', linestyles='--'); #vert line added (red)
plt.text(train['Yards'].mean()-18, plt.ylim()[1]-0.010, "Mean yards run = 4.21", size=12, color='r') #add sub-title (mean yards in red)
plt.xlabel("Yards")
plt.title("Yards Distribution", size=20);

In [ ]:
#Exploring correlation between (numerical )features - Looking for relationships based on "Yards" (nostra var target)
# NON SO QUANTO PUO' ESSERCI UTILE..DIREI UN GHEZZ COSì COME E'..
cor_mat = train.corr()
f, ax = plt.subplots(figsize=(20, 16))
plt.title('Correlation matrix of numerical Features', size =18)
sns.heatmap(cor_mat, vmax=.9, square=True, linewidths=.4 ,annot=True, cmap ='BuPu', fmt='.1g', annot_kws={'size':10});

In [ ]:
train_copy = train.copy() #copia del df

In [ ]:
## crearsi un sub-dataset in cui ci teniam solo le rows (players) oer le quali NflId è uguale alla NflIdRusher
# ovvero quello che "rusha" in attacco la palla (porta avanti l'azione di attacco con una corsa, per semplificarla)
train_copy.loc[train_copy.NflId==train_copy.NflIdRusher]  #23k righe da 590k originarie..

In [ ]:
train_copy_rush = train_copy.loc[train_copy.NflId==train_copy.NflIdRusher]  #23k righe da 590k originarie..
train_copy_rush.info(); #create a df with only 23k "rushing" plays

In [ ]:
#Exploring correlation between (numerical )features - Looking for relationships based on "Yards" (nostra var target)
#vediamo ora se cambia qualcosina.. (col df "rush")
cor_mat = train_copy_rush.corr()
f, ax = plt.subplots(figsize=(20, 16))
plt.title('Correlation matrix of numerical Features', size =18)
sns.heatmap(cor_mat, vmax=.9, square=True, linewidths=.4 ,annot=True, cmap ='BuPu', fmt='.1g', annot_kws={'size':10});

In [ ]:
#check missing values on the df
train_copy_rush.isnull().sum().sort_values(ascending=False)

Ok, here we have some NAs...although I think that the most (?) important to fix are those related to the OFF_Formation and DefensersInTheBox.

Variable  ------ Number of NAs
- WindDirection             3647
- WindSpeed                 3065
- Temperature               2206
- GameWeather               1984
- StadiumType               1497
- FieldPosition              292

These others...aybe we can fix FieldPosition (vars indicating whihc side of the field, the play is hapepning on (which team, basically..)

In [ ]:
# Exploring yards by "downs"
fig, ax = plt.subplots(nrows=3,figsize=(12,14))
sns.violinplot(data = train_copy_rush, x = 'Down', y = 'Yards', ax=ax[0]) #violinplot

sns.boxplot(data = train_copy_rush, x = 'Down', y = 'Yards', ax=ax[1]) #boxplot

#countplot to see the distribution by down for yard (we expect a lot of 1st rushing plays..)
sns.countplot(data = train_copy_rush, x = 'Down', ax=ax[2]) #boxplot

plt.show()

On avg, FIRST (and second) downs are the ones with a great number of yards gaines (watchout! in first downs we can also have a lot of yards lost!), while 4th down has less variability and less amount of yards gaines (mainly due to the game "situation"..majority of 4th downs used as rushing play are those with the lowest yards to run in order to reach the "new first down",,for example in 4th&1 or 4th & short situations)

In [ ]:
train_copy_rush['OffenseFormation'].value_counts()

In [ ]:
# vorrei fa una sorta di distribution plot per each OffenseFormation (sulle Yards) ma non riesco..
# avevo trovato uno con ggplot che filtrava e veniva bene...bbboh
g = sns.FacetGrid(train_copy_rush, col = "OffenseFormation")
g.map(plt.hist, "Yards")
plt.show()

In [ ]:
#Exploring Yards by Offense Formation (NON FUNZIONA COSì..mi dovrei creare one-hot encoding per ciascuna formazione/modalità..)


f, ax = plt.subplots(4, 2, figsize=(12, 12), sharex=True)
sns.distplot( train_copy_rush["OffenseFormation"=="SINGLEBACK"] , color="skyblue", ax=axes[0, 0])
sns.distplot( train_copy_rush["OffenseFormation"=='SHOTGUN'] , color="olive", ax=axes[0, 1])
sns.distplot( train_copy_rush["OffenseFormation"=='I_FORM'] , color="gold", ax=axes[1, 0])
sns.distplot( train_copy_rush["OffenseFormation"=='PISTOL'] , color="teal", ax=axes[1, 1])
sns.distplot( train_copy_rush["OffenseFormation"=='JUMBO'] , color="skyblue", ax=axes[2, 0])
sns.distplot( train_copy_rush["OffenseFormation"=='WILDCAT'] , color="olive", ax=axes[2, 1])
sns.distplot( train_copy_rush["OffenseFormation"=='EMPTY'] , color="gold", ax=axes[3, 0])
sns.distplot( train_copy_rush["OffenseFormation"=='ACE'] , color="teal", ax=axes[3, 1])

#sns.distplot(x = 'Down', y = 'OffenseFormation') #boxplot
